In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA

In [3]:
# Load the datasets
historical_data = pd.read_csv('../Data/2021-2022 Football Player Stats.csv', sep = ';', encoding_errors = 'ignore')
unclassified_data = pd.read_csv('../Data/player-data/player-combined-data.csv')

# Delete specified columns
columns_to_delete = ['home_team', 'away_team', 'year', 'Match', 'Competition', 'Date'] # Keeping 'team' and 'Position' columns
unclassified_data.drop(columns=columns_to_delete, inplace=True)

# Average each person's numeric columns to have one record per person
school_data = unclassified_data.groupby('player_name').agg(lambda x: x.iloc[0] if x.dtype == 'object' else x.mean()).reset_index()

# School player is 0
school_data['ps'] = 1

school_data

,player_name,team,Position,Minutes played,total_actions,successful_actions,goals,assists,shots,shots_on_target,...,gk_stat_shots_against,gk_stat_saves,gk_stat_reflex_saves,gk_stat_box_exits,gk_stat_passes_to_gk,gk_stat_passes_to_gk_completed,gk_stat_goal_kicks_attempted,gk_stat_short_goal_kicks,gk_stat_long_goal_kicks,ps
0,A. Adalsteinsson,Rutgers Scarlet Knights,"CB, LCB",91.142857,54.714286,40.500000,0.000000,0.000000,0.571429,0.214286,...,0.000000,0.000000,0.000000,0.000000,1.214286,1.214286,0.000000,0.000000,0.000000,1
1,A. Barger,Indiana Hoosiers,LB,91.447368,64.815789,41.052632,0.026316,0.026316,0.447368,0.157895,...,0.000000,0.000000,0.000000,0.000000,0.552632,0.552632,0.000000,0.000000,0.000000,1
2,A. Bilow,Ohio State Buckeyes,RW,44.276596,28.361702,17.127660,0.106383,0.063830,0.978723,0.489362,...,0.000000,0.000000,0.000000,0.000000,0.106383,0.106383,0.000000,0.000000,0.000000,1
3,A. Braman,Wisconsin Badgers,GK,74.400000,22.600000,17.800000,0.000000,0.000000,0.000000,0.000000,...,2.600000,2.200000,1.400000,0.800000,0.000000,0.000000,2.800000,0.400000,2.400000,1
4,A. Camara,Penn State Nittany Lion,"AMF, CF",69.320513,36.307692,15.153846,0.423077,0.051282,1.551282,0.730769,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Y. Senden,UCLA Bruins,CB,95.553846,85.061538,66.169231,0.015385,0.015385,0.200000,0.030769,...,0.000000,0.000000,0.000000,0.000000,1.446154,1.446154,0.000000,0.000000,0.000000,1
250,Z. Babiak,Michigan State Spartans,AMF,44.263158,25.789474,11.894737,0.052632,0.052632,0.473684,0.052632,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
251,Z. Kelly,Michigan State Spartans,GK,101.545455,30.818182,23.060606,0.000000,0.000000,0.000000,0.000000,...,4.151515,3.151515,2.060606,1.363636,0.000000,0.000000,7.484848,2.636364,4.848485,1
252,Z. Martens,Michigan Wolverines,"LDMF, RCMF, RDMF",41.791667,26.875000,16.958333,0.000000,0.000000,0.125000,0.083333,...,0.000000,0.000000,0.000000,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,1


In [4]:
# Delete specified columns
columns_to_delete2 = ['Rk', 'Nation', 'Comp']
historical_data.drop(columns=columns_to_delete2, inplace=True)

# Combine multiple 'Squad' values for the same person and average numeric values
clean_historical = historical_data.groupby('Player').agg(
    {
        'Squad': lambda x: ', '.join(set(x)),
        'Pos': 'first',
        **{col: 'mean' for col in historical_data.select_dtypes(include='number').columns}
    }
).reset_index()

# Professional is 0
clean_historical['ps'] = 0

clean_historical


,Player,Squad,Pos,Age,Born,MP,Starts,Min,90s,Goals,...,Crs,TklW,PKwon,PKcon,OG,Recov,AerWon,AerLost,AerWon%,ps
0,?tefan Radu,Lazio,DF,35.0,1986.0,10.0,6.0,556.0,6.20,0.00,...,1.130,0.81,0.0,0.00,0.0,7.74,2.10,1.77,54.20,0
1,?ukasz Fabia?ski,West Ham,GK,37.0,1985.0,37.0,37.0,3330.0,37.00,0.00,...,0.000,0.00,0.0,0.03,0.0,3.84,0.00,0.00,0.00,0
2,?ukasz Skorupski,Bologna,GK,31.0,1991.0,36.0,36.0,3240.0,36.00,0.00,...,0.000,0.00,0.0,0.06,0.0,4.50,0.00,0.03,0.00,0
3,Aarn Escandell,Granada,GK,26.0,1995.0,4.0,3.0,284.0,3.20,0.00,...,0.000,0.00,0.0,0.00,0.0,7.50,0.00,0.00,0.00,0
4,Aarn Martn,Mainz 05,DF,25.0,1997.0,28.0,25.0,2292.0,25.50,0.04,...,2.900,1.25,0.0,0.00,0.0,6.82,0.67,1.10,37.80,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2781,scar Rodrguez Arnaiz,"Getafe, Sevilla",FWMF,23.0,1998.0,14.5,6.5,680.0,7.55,0.00,...,1.665,1.55,0.0,0.00,0.0,6.01,0.85,1.27,39.55,0
2782,scar Trejo,Rayo Vallecano,MF,34.0,1988.0,32.0,24.0,2007.0,22.30,0.13,...,1.840,1.39,0.0,0.00,0.0,8.03,1.21,1.88,39.10,0
2783,scar Valentn,Rayo Vallecano,MF,27.0,1994.0,35.0,29.0,2569.0,28.50,0.00,...,0.070,2.32,0.0,0.04,0.0,11.90,1.54,2.35,39.60,0
2784,scar de Marcos,Athletic Club,DF,33.0,1989.0,22.0,19.0,1679.0,18.70,0.05,...,2.140,1.76,0.0,0.00,0.0,9.09,1.44,0.75,65.90,0


In [5]:


# List of columns from the first documentation
first_list = [
    "Player",
    "Squad",
    "Pos",              # Position on the field
    "Min",              # Minutes played
    "AerWon + AerLost", # Aerial duels attempted (AerWon + AerLost)
    "Tkl",              # Defensive duels attempted
    "TouAttPen",        # Touches inside the penalty area
    "Off",              # Offsides
    "CarProg",          # Progressive carries towards the opponent's goal
    "Fld",              # Fouls drawn
    "Goals",            # Goals scored
    "Assists",          # Assists made
    "Shots",            # Shots attempted
    "SoT",              # Shots on target
    "Recov",            # Ball recoveries
    "CarDis",           # Losses of possession
    "Int",              # Interceptions
    "CrdR",             # Red cards received
    "CrdY",             # Yellow cards received
    "Fls",              # Fouls committed
    "PasTotAtt",        # Total passes attempted
    "PasLonAtt",        # Long passes attempted
    "PasProg",          # Forward passes moving towards the opponent's goal
    "Pas3rd",           # Passes into final third
    "PPA",              # Passes into the penalty area
    "Rec",              # Passes received
    "PasCrs",           # Crosses attempted
    "CrsPA",            # Accurate crosses
    "PasAss",           # Shot assists
    "ps"
]

# List of columns from the second documentation
second_list = [
    "player_name",
    "team",
    "Position",                    # Position on the field
    "Minutes played",              # Minutes played
    "aerial_duels",                # Aerial duels attempted
    "defensive_duels",             # Defensive duels attempted
    "touches_inside_box",          # Touches inside the penalty area
    "offsides",                    # Offsides
    "progressive_runs_attempted",  # Progressive runs towards the opponent's goal
    "fouls_drawn",                 # Fouls drawn
    "goals",                       # Goals scored
    "assists",                     # Assists made
    "shots",                       # Shots attempted
    "shots_on_target",             # Shots on target
    "recoveries",                  # Ball recoveries
    "losses",                      # Losses of possession
    "interceptions",               # Interceptions
    "red_cards",                   # Red cards received
    "yellow_cards",                # Yellow cards received
    "fouls",                       # Fouls committed
    "total_passes",                # Total passes attempted
    "long_passes",                 # Long passes attempted
    "forward_passes",              # Forward passes moving towards the opponent's goal
    "passes_into_final_third",     # Passes into final third
    "passes_into_box",             # Passes into the penalty area
    "received_passes",             # Passes received
    "crosses",                     # Crosses attempted
    "accurate_crosses",            # Accurate crosses
    "shot_assists",                # Shot assists
    "ps"
]

# Common names for the columns
common_names = [
    "Player",
    "Team"
    "Position",                  # Position on the field
    "Minutes Played",            # Minutes played
    "Aerial Duels Attempted",    # Aerial duels attempted
    "Defensive Duels Attempted", # Defensive duels attempted
    "Touches in Penalty Area",   # Touches inside the penalty area
    "Offsides",                  # Offsides
    "Progressive Runs",          # Progressive runs towards the opponent's goal
    "Fouls Drawn",               # Fouls drawn
    "Goals",                     # Goals scored
    "Assists",                   # Assists made
    "Shots",                     # Shots attempted
    "Shots on Target",           # Shots on target
    "Ball Recoveries",           # Ball recoveries
    "Losses of Possession",      # Losses of possession
    "Interceptions",             # Interceptions
    "Red Cards",                 # Red cards received
    "Yellow Cards",              # Yellow cards received
    "Fouls Committed",           # Fouls committed
    "Total Passes Attempted",    # Total passes attempted
    "Long Passes Attempted",     # Long passes attempted
    "Forward Passes",            # Forward passes moving towards the opponent's goal
    "Passes into Final Third",   # Passes into final third
    "Passes into Penalty Area",  # Passes into the penalty area
    "Passes Received",           # Passes received
    "Crosses Attempted",         # Crosses attempted
    "Accurate Crosses",          # Accurate crosses
    "Shot Assists",              # Shot assists
    "ps"
]

# Create dictionaries to rename the columns
first_rename_dict = dict(zip(first_list, common_names))
second_rename_dict = dict(zip(second_list, common_names))

# Handle the combined column 'AerWon + AerLost' in the first DataFrame
clean_historical['AerWon + AerLost'] = clean_historical['AerWon'] + clean_historical['AerLost']

# Select and rename columns from the first DataFrame
historical_data_subset = clean_historical[first_list].copy()
historical_data_subset.rename(columns=first_rename_dict, inplace=True)

# Select and rename columns from the second DataFrame
school_data_subset = school_data[second_list].copy()
school_data_subset.rename(columns=second_rename_dict, inplace=True)

# Combine the two DataFrames
combined_df = pd.concat([historical_data_subset, school_data_subset], ignore_index=True)



In [6]:
combined_df

,Player,TeamPosition,Minutes Played,Aerial Duels Attempted,Defensive Duels Attempted,Touches in Penalty Area,Offsides,Progressive Runs,Fouls Drawn,Goals,...,Long Passes Attempted,Forward Passes,Passes into Final Third,Passes into Penalty Area,Passes Received,Crosses Attempted,Accurate Crosses,Shot Assists,ps,ps
0,?tefan Radu,Lazio,DF,556.000000,3.870000,1.290000,0.650000,0.000000,2.580000,0.970000,...,52.600000,5.480000,2.580000,3.390000,0.000000,38.100000,1.130000,0.000000,0.160000,0
1,?ukasz Fabia?ski,West Ham,GK,3330.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.190000,...,27.200000,19.000000,0.000000,0.300000,0.000000,11.100000,0.000000,0.000000,0.030000,0
2,?ukasz Skorupski,Bologna,GK,3240.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.170000,...,35.000000,18.900000,0.000000,0.110000,0.000000,16.400000,0.000000,0.000000,0.000000,0
3,Aarn Escandell,Granada,GK,284.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.310000,...,39.100000,29.700000,0.000000,0.940000,0.000000,11.300000,0.000000,0.000000,0.000000,0
4,Aarn Martn,Mainz 05,DF,2292.000000,1.770000,2.270000,0.940000,0.000000,2.310000,0.670000,...,43.800000,12.100000,3.020000,2.120000,1.100000,28.500000,2.900000,0.710000,1.920000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,Y. Senden,UCLA Bruins,CB,95.553846,2.876923,5.969231,0.307692,0.000000,2.600000,0.415385,...,64.907692,10.800000,26.707692,11.400000,1.261538,47.138462,0.384615,0.184615,0.215385,1
3036,Z. Babiak,Michigan State Spartans,AMF,44.263158,3.263158,1.789474,1.105263,0.315789,0.368421,0.631579,...,10.052632,0.315789,1.421053,1.000000,0.315789,7.684211,0.421053,0.105263,0.157895,1
3037,Z. Kelly,Michigan State Spartans,GK,101.545455,0.606061,0.151515,0.000000,0.000000,0.000000,0.151515,...,19.060606,6.909091,8.666667,2.000000,0.060606,11.787879,0.000000,0.000000,0.000000,1
3038,Z. Martens,Michigan Wolverines,"LDMF, RCMF, RDMF",41.791667,1.083333,3.541667,0.083333,0.000000,0.333333,0.291667,...,16.333333,1.791667,5.666667,2.125000,0.500000,9.541667,0.208333,0.083333,0.166667,1


In [7]:
clean_historical.to_csv('clean_combined_df.csv', index=True)